In [ ]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

from collections import defaultdict
from itertools import tee
from datetime import datetime
from base64 import b64encode

In [ ]:
base_url = 'https://admin.psylio.com/assistance-requests'

email = 'email@provider.com'
password = 'your_password_here'

In [ ]:
cookies = {
    'assistance-requests-order': 'eyJpdiI6IklMb2ZpeW1scWxqVzJXb0VoRUptYXc9PSIsInZhbHVlIjoidzFpWWRuQ2tPT2E0TlZjZW42SXpMZz09IiwibWFjIjoiNjc2NjA0NWQwMjgyODQ2MzNmMjdiNmMzYWJhZmIzYjJiZjEwMjM3YWMwY2E1MGFjZTZkNmNjMDY5ZGVmMTYxYyJ9',
    'assistance-requests-direction': 'eyJpdiI6IlVmWXgzK3B2YVp5c01wZ0UySVwvU3VBPT0iLCJ2YWx1ZSI6ImVYZUt4R21IVkdWbUFNenRPbmFKSlE9PSIsIm1hYyI6IjJjOTc3ZTQyMTNlZGU3ZjMzZTE0ZTQwZTcxODA0MDkwNjkwNDNmNTljMWYxYWNhZDkzMjQ2NDgyYzMzOWFjMjcifQ%3D%3D',
    'XSRF-TOKEN': 'eyJpdiI6InJmNG9LWjhtbE9lWnh3RVA0RW1EdWc9PSIsInZhbHVlIjoiUGtPbFdGeXFrS2t5OXMyaTZ1djBqc3ZkRE5yWDlLTEpjeWNkemM3b2JQWmNVMkMxWjE0K0lwa3d1Q25zUHNVViIsIm1hYyI6ImQ1YTIyZTc2MTMxYTg5NWMwMzJmODQyYzhkMDBjOTFlNzhhMzU2MGVlZWEyZjE5MDgwNWQ0YmM4ZTUxN2M0NDYifQ%3D%3D',
    'epsylio_admin_session': 'eyJpdiI6IkJ0MEh3NzBCU2sxd21hb1VcL0E0MjZRPT0iLCJ2YWx1ZSI6IlNha2dqREpUXC9kekNRcU5VMEdCT0dHRGxYREtCREZTaDdENXBvTUtVdTVLVG5ucTlQMml0UWdvU0VOTWRka2w0IiwibWFjIjoiYjNjYjVlOTFjN2UwYWYzMjIzZjUxNzBhZjhlMWE1MzliMzNmZWU0YmRlNzI5NTg0OTJlNWI4MWFhYTJlMTFlOSJ9',
}

headers = {
    'cookie': '; '.join([f'{k}={v}' for k, v in cookies.items()]),
}

In [ ]:
def open_html(template, filename='tmp.html'):
    import webbrowser
    import os
  
    with open(filename, 'wb') as f:
        # writing the code into the file
        f.write(template)
      
    # 1st method how to open html files in chrome using
    filename = f'file:///{os.getcwd()}/{filename}'
    webbrowser.open_new_tab(filename)

In [ ]:
def login(headers,
          url='https://admin.psylio.com/login',
          email='email@provider.com',
          password='your_password_here'):

    payload = {
        '_token': '5zppYCK0LnxhlY6JvCRGgHXjFpp6X5rkh7qbKVn1',
        'login': {
            'email': email,
            'password': password,
            'account': 'ca',
        },
    }
   
    resp = requests.post(url, headers=headers, data=payload)
    assert resp.status_code == 200, 'Login failed :('
    


print('Attempting login...', end=' ')
login(headers)
print('Login successful!')

In [ ]:
def get_records(headers):
    url = 'https://admin.psylio.com/assistance-requests'
    resp = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(resp.content, 'html.parser')
    tbody = soup.find('table').tbody
    
    client_ids = []
    for row in tbody.find_all('tr'):
        cells = row.find_all('td')
        cid = cells[1].a.get('href').split('/')[4]
        client_ids.append(cid)
    
    records_df = pd.read_html(resp.text)[0]
    records_df.drop(columns=['Unnamed: 4'], inplace=True)
    records_df['client_id'] = client_ids
    
    return records_df

In [ ]:
records_df = get_records(headers)
records_df.head()

In [ ]:
def create_invoice(headers, client_id):
    url = f'{base_url}/{client_id}/invoices/create'
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
    forms = soup.find_all('form')

    payload = {}

    for field in forms[1].find_all('input'):
        field_name, field_value = field.get('name'), field.get('value', '')
        if field_name is not None:
            add_to_payload(payload, field_name, field_value)

    for field in forms[1].find_all('textarea'):
        field_name, field_value = field.get('name'), field.get_text()
        add_to_payload(payload, field_name, field_value)

    payload['institution'] = {'id': ''}
    payload['paymentDate'] = str(datetime.today()).split()[0]
    payload['paymentTypes'] = ','.join(['debit_transfer'])
    payload['items']['0'].update({
        'service': 'Suivi client',
        'price_per_unit': payload['items']['0']['price_per_unit'].replace(',', '.'),
    })
    
    new_headers = headers.copy()
    new_headers['Content-Type'] = 'application/json'

    resp = requests.post(invoice_url, headers=new_headers, data=json.dumps(payload))
    print(resp.status_code)

    open_html(resp.content)

        
def add_to_payload(payload, field_name, field_value, pattern=r'(\[|\]|\]\[])'):
    keys = re.sub(pattern, ' ', field_name).split()
    if len(keys) == 1:
        payload[field_name] = field_value
    else:
        current = payload
        for k1, k2 in pairwise(keys):
            if k1 in current:
                current = current[k1]
            else:
                current[k1] = {}
                current = current[k1]

        current[k2] = field_value
    
    
def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [ ]:
create_invoice(headers, records_df.loc[0]['client_id'])

In [ ]:
def get_receipts(headers, client_id):
    url = f'{base_url}/{client_id}/invoices?state=paid'
    resp = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(resp.content, 'html.parser')
    tbody = soup.find('table').tbody
    
    receipt_ids = []
    for row in tbody.find_all('tr'):
        cells = row.find_all('td')
        rid = cells[0].a.get('href').split('/')[6]
        receipt_ids.append(rid)
    
    receipts_df = pd.read_html(resp.text)[0]
    receipts_df.drop(columns=['Actions'], inplace=True)
    receipts_df['receipt_id'] = receipt_ids
    
    return receipts_df

In [ ]:
client_id = records_df.loc[0]['client_id']
receipts_df = get_receipts(headers, client_id)
receipts_df.head()

In [ ]:
def send_receipt(headers, client_id, receipt_id):
    url = f'{base_url}/{client_id}/invoices/{receipt_id}'
    resp = requests.get(url, headers=headers)